In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd

In [2]:
# 드라이버 실행
driver = webdriver.Chrome()

In [3]:
# 기업 리스트 가져오기 & reshaping

jobplanet_excel = pd.read_excel("잡플래닛_기업리스트.xlsx", index_col = 0)

jobplanet_reshape = pd.melt(jobplanet_excel)
jobplanet_reshape.head()

In [ ]:
# 캐치 URL 가져오기 & 수정

# part1
data = pd.read_csv("urls_null.csv", sep=",")
part1 = pd.Series(list(data))
part1[198] = 'https://www.catch.co.kr/Comp/CompSummary/IW7328'
part1[443] = '방송통신대학교'
# part1

# part2
data = pd.read_csv("urls_2.csv", sep=",")
pd.melt(data).value
part2 = pd.melt(data)
# part2.value[443:]
# jobplanet_reshape[443:]
# part2.value

In [ ]:
# 리뷰 페이지로 이동
index = 0
for i in part1:
    if i[:4] == 'http':
        part1[index] = i.replace('CompSummary', 'ReviewInfo')
    index += 1   
#print(part1)

index = 0
for i in part2.value:
    if i[:4] == 'http':
        part2.value[index] = i.replace('CompSummary', 'ReviewInfo')
    index += 1    
#print(part2.value)

total = pd.concat([part1, part2.value], axis = 0, ignore_index = True)
#total

In [4]:
# 잡플래닛 URL 가져오기 

data = pd.read_csv("jobplanet_urls.csv", sep=",")
pd.melt(data).value
jobplanet_urls_2 = pd.melt(data)
#jobplanet_urls_2

data = pd.read_csv("jobplanet_urls - 73.csv", sep=",")
pd.melt(data).value
jobplanet_urls_1 = pd.melt(data)

jobplanet_urls = pd.concat([jobplanet_urls_1, jobplanet_urls_2], axis = 0, ignore_index = True)
jobplanet_urls

,variable,value
0,0,https://www.jobplanet.co.kr/companies/70749/in...
1,1,https://www.jobplanet.co.kr/companies/3616/inf...
2,2,https://www.jobplanet.co.kr/companies/78632/in...
3,3,https://www.jobplanet.co.kr/companies/49174/in...
4,4,https://www.jobplanet.co.kr/companies/87279/in...
...,...,...
995,995,https://www.jobplanet.co.kr/companies/89482/in...
996,996,https://www.jobplanet.co.kr/companies/93708/in...
997,997,https://www.jobplanet.co.kr/companies/90908/in...
998,998,https://www.jobplanet.co.kr/companies/89556/in...


## 로고 URL 수집

In [87]:
# 잡플래닛 로고 URL 수집

name = []
image = []

index = 0

for i in jobplanet_urls.value:
    if i[0:4] != 'http':
        index += 1
        name.append(jobplanet_reshape.value[index])
        image.append(None)
    else:
        index += 1
        driver.get(i)
        time.sleep(5)
        name.append(jobplanet_reshape.value[index])
        image.append(driver.find_element_by_css_selector('body > div.body_wrap > div.cmp_hd > div.new_top_bnr > div > div.top_bnr_wrap > div > div > div.company_info_sec > div.company_logo_box > a > span > img').get_attribute('src'))

total_data = pd.DataFrame()

total_data['회사명'] = pd.Series(name)
total_data['로고URL'] = pd.Series(image)

total_data.to_excel("로고URL.xlsx", index=True)

## 복지정보 수집

In [9]:
# 잡플래닛 URL 복지 탭으로 변경

index = 0
for i in jobplanet_urls.value:
    if i[:4] == 'http':
        jobplanet_urls.value[index] = i.replace('info', 'benefits')
    index += 1   
    
jobplanet_urls_benefits = jobplanet_urls.value.copy()
print(jobplanet_urls_benefits)

0      https://www.jobplanet.co.kr/companies/70749/be...
1      https://www.jobplanet.co.kr/companies/3616/ben...
2      https://www.jobplanet.co.kr/companies/78632/be...
3      https://www.jobplanet.co.kr/companies/49174/be...
4      https://www.jobplanet.co.kr/companies/87279/be...
                             ...                        
995    https://www.jobplanet.co.kr/companies/89482/be...
996    https://www.jobplanet.co.kr/companies/93708/be...
997    https://www.jobplanet.co.kr/companies/90908/be...
998    https://www.jobplanet.co.kr/companies/89556/be...
999    https://www.jobplanet.co.kr/companies/13123/be...
Name: value, Length: 1000, dtype: object


In [206]:
# 잡플래닛 복지 코멘트 수집
import math

name = []
cate = []
comm = []
score = []

index = 0

for i in jobplanet_urls.value:
    if i[0:4] != 'http':
        index += 1
        name.append(jobplanet_reshape.value[index])
        cate.append(None)
        comm.append(None)
        score.append(None)
    else:
        index += 1
        driver.get(i)
        time.sleep(5)
        
        pages = math.ceil(int(driver.find_element_by_css_selector(f'#mainContents > div:nth-child(7) > div > div.comment_box-hd > span > span').text)/20)
        for p in range(1, pages+1):
            reviews = len(driver.find_elements_by_css_selector('#mainContents > div:nth-child(7) > div > div.comment_box-bd > .comment_wrap'))
            for nth in range(1, reviews+1):
                name.append(jobplanet_reshape.value[index])
                cate.append(driver.find_element_by_css_selector(f'#mainContents > div:nth-child(7) > div > div.comment_box-bd > div:nth-child({nth}) > a').text)
                comm.append(driver.find_element_by_css_selector(f'#mainContents > div:nth-child(7) > div > div.comment_box-bd > div:nth-child({nth}) > p').text)
                score.append(driver.find_element_by_css_selector(f'#mainContents > div:nth-child(7) > div > div.comment_box-bd > div:nth-child({nth}) > span > em').text)
            try: driver.get(f'{i}?page={p+1}')
            except: pass

total_data = pd.DataFrame()

total_data['회사명'] = pd.Series(name)
total_data['항목'] = pd.Series(cate)
total_data['코멘트'] = pd.Series(comm)
total_data['평점'] = pd.Series(score)

total_data.to_excel("잡플래닛_복지리뷰코멘트.xlsx", index=True)

In [25]:
# 잡플래닛 복지 평점 수집

name = []
a = []
b = []
c = []
d = []
e = []

index = -1

for i in jobplanet_urls_benefits:
    if i[0:4] != 'http':
        index += 1
        name.append(jobplanet_reshape.value[index])
        a.append(None)
        b.append(None)
        c.append(None)
        d.append(None)
        e.append(None)
    else:
        index += 1
        driver.get(i)
        time.sleep(3)
        
        try: 
            name.append(jobplanet_reshape.value[index])
            a.append(driver.find_element_by_css_selector(f'#mainContents > div:nth-child(1) > div > div.summary_bd > div.chart_bar_vertical > div:nth-child(1) > div > span').text)
            b.append(driver.find_element_by_css_selector(f'#mainContents > div:nth-child(1) > div > div.summary_bd > div.chart_bar_vertical > div:nth-child(2) > div > span').text)
            c.append(driver.find_element_by_css_selector(f'#mainContents > div:nth-child(1) > div > div.summary_bd > div.chart_bar_vertical > div:nth-child(3) > div > span').text)
            d.append(driver.find_element_by_css_selector(f'#mainContents > div:nth-child(1) > div > div.summary_bd > div.chart_bar_vertical > div:nth-child(4) > div > span').text)
            e.append(driver.find_element_by_css_selector(f'#mainContents > div:nth-child(1) > div > div.summary_bd > div.chart_bar_vertical > div:nth-child(5) > div > span').text)
        except:
            name.append(jobplanet_reshape.value[index])
            a.append(None)
            b.append(None)
            c.append(None)
            d.append(None)
            e.append(None)
            

total_data = pd.DataFrame()

total_data['회사명'] = pd.Series(name)
total_data['매우불만족'] = pd.Series(a)
total_data['불만족'] = pd.Series(b)
total_data['보통'] = pd.Series(c)
total_data['만족'] = pd.Series(d)
total_data['매우만족'] = pd.Series(e)

total_data.to_excel("잡플래닛_복지평점.xlsx", index=True)